DATA SCIENTIST CAPSTONE PROJECT - PREPARE AND CLEAN DATA

Main goal of this part is to prepare the data to begin the analysis
.
As a input, there are two types of excel files: One with transactional data that is generated every day, and a second ond with master data.
To analyse the data it is necessary to group all the columns in one data frame.

All the columns are not needed, so most of them will be ignored and I keep only the ones interesting for the analysis
I only keep the rows that have values, because represent a process completed from the begining to the end. Rows with NaN values are eliminated.

Is necessary also to filter the rows that are in the scope. This means the ones that have a PLP value representative.



In [2]:
# Import libraries

import sys
import pandas as pd
import numpy as np
import glob
import sqlite3

Import the Data Files

Here we have two types
- Transactional data with excel: Several files according to an initial and final date (4 first weeks October 2025)
- Master date with excel: One file

In [3]:
# Import all the Excel Files that are in the data folder (Transactional data)

excel_files = glob.glob("data/*.xlsx")
df_transactional_all_data = pd.concat([pd.read_excel(f) for f in excel_files], ignore_index=True)

# Display the first rows
df_transactional_all_data.head()

,SFab,GLin,UbiLínea,Alm.,Material,Status,Denominación,Tipo Sum,Cap.Cont,N.bult,...,SM,F.Conf.Toma,H.Conf.Toma,F.Conf OT,H.Conf OT,H.Demora,Usu.Conf,Creador,Ubic.proc.,Tp.alm.proc.
0,402,45,410P2_NO,ST12,W06C0035621A,1.0,CAJAS DE BAJOS,NO,30,1,...,A,29.09.25,20:18,NaN,NaN,9,NaN,SCOMPTE,PASILLO6,AGL
1,431,99,C17D..NO,ST12,W02Q0407255AD,1.0,APOYO OSCILAN,NO,60,1,...,A,NaN,NaN,NaN,NaN,9,NaN,NaN,NaN,NaN
2,402,0I,004P2_NO,ST12,W05FA863459A 4PK,2.0,RECUBRIMIENTO,NO,36,1,...,A,29.09.25,22:46,NaN,NaN,7,NaN,RUIZP4,PASILLO1,AGL
3,402,41,321P4_WN,ST12,W05FA867765B 4PK,2.0,REVEST.PASARRUEDAS,WN,40,1,...,A,29.09.25,07:36,NaN,NaN,22,NaN,MARCO01,PASILLO6,AGL
4,401,22,C09P1_NO,ST12,W05TA839016Q,1.0,CERRADURA D/PUERTA,NO,240,1,...,A,29.09.25,18:47,NaN,NaN,11,NaN,ONORATO01,PASILLO5,AGL


In [4]:
# Import Excel file with Master data

# Path to the file
file_path = "master_data/Parasum_iTLS.xlsx"

# Read the Excel file
df_master_data = pd.read_excel(file_path)

# Display the first rows
print(df_master_data.head())


     Material   SM                 Denominación  SFab GLin  UbiLínea  Nº  \
0  46471120X1  1.0    LAPIZ RETOQUES DESIRE RED   404   05  B16I..AL   1   
1  4647112511  1.0     LAPIZ RETOQUES STAHLBLAU   404   05  B16I..AL   1   
2  4647112B9A  1.0  LAPIZ RETOQUES BLANCO CANDY   404   05  B16I..AL   1   
3  4647112M6T  1.0     L-PIZ RETOQUES GREEN MET   404   05  B16I..AL   1   
4  4647112N7H  1.0     LAPIZ RETOQUES GREY MATT   404   05  B16I..AL   1   

  Cont.Básic Cont.Sumin TSum  ...  Válido de             Válido a Cap.Básica  \
0        NaN    0005SCH   NX  ... 2000-01-01  9999-12-31 00:00:00          0   
1        NaN    0005SCH   NX  ... 2000-01-01  9999-12-31 00:00:00          0   
2    0005SCH    0005SCH   NX  ... 2000-01-01  9999-12-31 00:00:00          1   
3        NaN    0005SCH   NX  ... 2000-01-01  9999-12-31 00:00:00          0   
4        NaN    0005SCH   NX  ... 2000-01-01  9999-12-31 00:00:00          0   

   Cap. Sumin  NºMax.Cont  Num. Ref.  SumEmb LibrUtiliz Cont. 

Merge Data

The 2 files have complementary columns that should be used in the analysis and prediction model. It is necessary to group them in one data frame

In [27]:
# Merge the 2 data frames using the Key colums: SFab, GLin, UbiLínea and Material. Both columns are in the 2 file types.
df_manual_requests = pd.merge(
    df_transactional_all_data,
    df_master_data,
    on=["SFab", "GLin", "UbiLínea", "Material"],
    how="left"  # or "inner" depending on your needs
)

df_manual_requests.head()

,SFab,GLin,UbiLínea,Alm._x,Material,Status,Denominación_x,Tipo Sum,Cap.Cont,N.bult,...,Válido de,Válido a,Cap.Básica,Cap. Sumin,NºMax.Cont,Num. Ref.,SumEmb,LibrUtiliz,Cont. Transp1,Cap. Transp1
0,402,45,410P2_NO,ST12,W06C0035621A,1.0,CAJAS DE BAJOS,NO,30,1,...,2000-01-01,9999-12-31 00:00:00,0.0,30.0,2.0,358451.0,NaN,1200.0,NaN,0.0
1,431,99,C17D..NO,ST12,W02Q0407255AD,1.0,APOYO OSCILAN,NO,60,1,...,2000-01-01,9999-12-31 00:00:00,60.0,60.0,2.0,357935.0,NaN,240.0,111960,240.0
2,402,0I,004P2_NO,ST12,W05FA863459A 4PK,2.0,RECUBRIMIENTO,NO,36,1,...,2000-01-01,9999-12-31 00:00:00,36.0,36.0,2.0,346884.0,NaN,108.0,114888,36.0
3,402,41,321P4_WN,ST12,W05FA867765B 4PK,2.0,REVEST.PASARRUEDAS,WN,40,1,...,2000-01-01,9999-12-31 00:00:00,40.0,40.0,2.0,357897.0,NaN,2240.0,114888,40.0
4,401,22,C09P1_NO,ST12,W05TA839016Q,1.0,CERRADURA D/PUERTA,NO,240,1,...,2000-01-01,9999-12-31 00:00:00,16.0,240.0,2.0,359820.0,X,10320.0,VWPAL,240.0


Filter Data 1 - Columns

Data frame for analysis is too big. It has a lot of columns that are no necessary.
Note: All the column names are in Spanish.

In [28]:
# Subset the data frame with only the necessary columns for the analysis

# List of columns to keep
columns_to_keep = [
    "SFab","GLin","UbiLínea", "Material", "Denominación_x", "Status", "Tipo Sum", "Consumo",
    "F.Creac", "H.Creac", "F.Conf OT", "H.Conf OT", "Ubic.proc.",
    "Tp.alm.proc.", "UbicDest", "ÁrSumProd.", "Válido de", "Válido a", "Cap. Sumin"
]

# Subset the DataFrame
df_manual_requests = df_manual_requests[df_manual_requests.columns.intersection(columns_to_keep)]

df_manual_requests.head()

,SFab,GLin,UbiLínea,Material,Status,Denominación_x,Tipo Sum,Consumo,F.Creac,H.Creac,F.Conf OT,H.Conf OT,Ubic.proc.,Tp.alm.proc.,UbicDest,ÁrSumProd.,Válido de,Válido a,Cap. Sumin
0,402,45,410P2_NO,W06C0035621A,1.0,CAJAS DE BAJOS,NO,109.333,29.09.25,20:16:22,NaN,NaN,PASILLO6,AGL,10DI_410P2,10L2_DI,2000-01-01,9999-12-31 00:00:00,30.0
1,431,99,C17D..NO,W02Q0407255AD,1.0,APOYO OSCILAN,NO,975.333,29.09.25,20:21:24,NaN,NaN,NaN,NaN,TMEL03_G99,TMEL3_,2000-01-01,9999-12-31 00:00:00,60.0
2,402,0I,004P2_NO,W05FA863459A 4PK,2.0,RECUBRIMIENTO,NO,37.333,29.09.25,22:41:52,NaN,NaN,PASILLO1,AGL,10SI_004P2,10L2_SI,2000-01-01,9999-12-31 00:00:00,36.0
3,402,41,321P4_WN,W05FA867765B 4PK,2.0,REVEST.PASARRUEDAS,WN,636.667,29.09.25,07:32:23,NaN,NaN,PASILLO6,AGL,10CI_321P4,10L2_CIW,2000-01-01,9999-12-31 00:00:00,40.0
4,401,22,C09P1_NO,W05TA839016Q,1.0,CERRADURA D/PUERTA,NO,601.667,29.09.25,18:42:25,NaN,NaN,PASILLO5,AGL,08CD_C09P1,08L2_CD,2000-01-01,9999-12-31 00:00:00,240.0


In [29]:
# Check the matrix size
df_manual_requests.shape

(509908, 19)

Clean Data: Drop Missing Values

Rows with NaN are useless, because represent incompleted processes.


In [30]:
# Drop rows with any NaN values
df_manual_requests = df_manual_requests.dropna()

df_manual_requests.head()

,SFab,GLin,UbiLínea,Material,Status,Denominación_x,Tipo Sum,Consumo,F.Creac,H.Creac,F.Conf OT,H.Conf OT,Ubic.proc.,Tp.alm.proc.,UbicDest,ÁrSumProd.,Válido de,Válido a,Cap. Sumin
134,900,H2,IS_04_0_,W05H2800345 S7G,1.0,MANECILLA PUERTA,SJ,2.667,29.09.25,00:29:59,29.09.25,00:30,11-16-13,EST,IS_04_0_,IS_04_0_,2000-01-01,9999-12-31 00:00:00,108.0
135,900,H3,IS_04_0_,W06F2800345A S9R,1.0,MANECILLA PUERTA,SJ,7.333,29.09.25,00:30:42,29.09.25,00:30,08-07-11,EST,IS_04_0_,IS_04_0_,2000-01-01,9999-12-31 00:00:00,34.0
141,900,E3,A2_1_38,W06F0867288B 82V,1.0,REVEST.MONTANTE C,JA,168.667,29.09.25,00:04:11,29.09.25,00:32,PASILLO1,AGL,A2_1_38,IS_02_1_,2000-01-01,9999-12-31 00:00:00,44.0
155,900,J3,IS_06_0_,W00AJ300044D,1.0,3352 CAMBIO,JA,120.333,29.09.25,00:31:26,29.09.25,00:36,SE-179,C1C,IS_06_0_,IS_06_0_,2000-01-01,9999-12-31 00:00:00,6.0
166,900,G3,A3_1_03,W02Q0816743AK,1.0,TUBO MEDIO REFRIG.,JA,46.333,29.09.25,00:00:53,29.09.25,00:37,PASILLO2,AGL,A3_1_03,IS_03_01,2000-01-01,9999-12-31 00:00:00,50.0


In [31]:
# Check the matrix size
df_manual_requests.shape

(313510, 19)

Filter Data 2 - Rows


Not all the data provided is usefull for the analysis.
Scope:

Tipo Sum: NO and WN

SFab: 401, 402, 441 and 431

ÁrSumProd.: Should begin with "10", "09" or "08"

In [32]:

df_manual_requests_scope = df_manual_requests[
    (df_manual_requests["Tipo Sum"].isin(["NO", "WN"])) &
    (df_manual_requests["SFab"].isin([401, 402, 441, 431])) &
    (df_manual_requests["ÁrSumProd."].astype(str).str.startswith(("10", "09", "08")))
]

# Rename columns to better understand the analysis
df_manual_requests_scope = df_manual_requests_scope.rename(columns={
    'Denominación_x': 'Denominacion',
    'UbicDest': 'PLP',
    'ÁrSumProd.': 'PVB'
})

# Check the matrix size and first rows
print(df_manual_requests_scope.shape)
df_manual_requests_scope.head()

(57082, 19)


,SFab,GLin,UbiLínea,Material,Status,Denominacion,Tipo Sum,Consumo,F.Creac,H.Creac,F.Conf OT,H.Conf OT,Ubic.proc.,Tp.alm.proc.,PLP,PVB,Válido de,Válido a,Cap. Sumin
285,431,95,049P3_NO,W02Q0825671D,1.0,PROT.CAL.TUN.CENT.,NO,780.667,29.09.25,00:24:48,29.09.25,00:52,BL-152,C1C,09ED_049P3,09MEF_ED,2000-01-01,9999-12-31 00:00:00,100.0
293,431,95,051P1_NO,W02Q0825701A,1.0,PROT.CAL.TRAM.POS.,NO,780.667,29.09.25,00:25:00,29.09.25,00:53,PASILLO3,AGL,09ED_054P1,09MEF_ED,2000-01-01,9999-12-31 00:00:00,180.0
308,441,9C,020P2_NO,W06F0837432G 5AP,1.0,GUIA P/VENTANA,NO,357.000,29.09.25,00:25:48,29.09.25,00:55,PASILLO1,AGL,09SC_020P2,09L3_SC,2000-01-01,9999-12-31 00:00:00,80.0
345,441,9B,013P1_NO,W06F0837431G 5AP,1.0,GUIA P/VENTANA,NO,357.000,29.09.25,00:23:07,29.09.25,00:58,PASILLO7,AGL,09SB_013P1,09L3_SB,2000-01-01,9999-12-31 00:00:00,80.0
452,441,9C,016P1_NO,W06F9839432L 5AP,1.0,GUIA P/VENTANA,NO,195.333,29.09.25,00:25:58,29.09.25,01:09,PASILLO5,AGL,09SC_016P1,09L3_SC,2000-01-01,9999-12-31 00:00:00,90.0


Add calculated Data

Now that all the basic data in the data frame is available, it is necessary to begin the calculations to add data needed for the analysis and the prediction model.

Supply Time: Duration in hours between the start of the event (Material Request) to the end of the event (Material Delivered)
Time between MatReq: Duration in time between the first Material Request and the Second Material Request

In [35]:
# Make a copy to avoid SettingWithCopyWarning
df = df_manual_requests_scope.copy()

# Create two new col's in order to store the datetime in a format that can be used
# Specify the format (example: YYYY-MM-DD for date, HH:MM:SS for time)
df['datetime_creac'] = pd.to_datetime(df['F.Creac'] + ' ' + df['H.Creac'], format='%d.%m.%y %H:%M:%S', errors='coerce')
df['datetime_conf'] = pd.to_datetime(df['F.Conf OT'] + ' ' + df['H.Conf OT'], format='%d.%m.%y %H:%M', errors='coerce')


# Calculate Supply Time
df['Supply_time'] = df['datetime_conf'] - df['datetime_creac']

# Calculate Supply Time in hours
df['Supply_time_hours'] = df['Supply_time'].dt.total_seconds() / 3600

print(df[['datetime_creac', 'datetime_conf', 'Supply_time', 'Supply_time_hours']].head(10))

# Save stats_df to Excel for internal analysis
df.to_excel('SPA_Tiempos_entre_Peticion_y_Entrega_a_punto_consumo.xlsx', index=False)



         datetime_creac       datetime_conf     Supply_time  Supply_time_hours
285 2025-09-29 00:24:48 2025-09-29 00:52:00 0 days 00:27:12           0.453333
293 2025-09-29 00:25:00 2025-09-29 00:53:00 0 days 00:28:00           0.466667
308 2025-09-29 00:25:48 2025-09-29 00:55:00 0 days 00:29:12           0.486667
345 2025-09-29 00:23:07 2025-09-29 00:58:00 0 days 00:34:53           0.581389
452 2025-09-29 00:25:58 2025-09-29 01:09:00 0 days 00:43:02           0.717222
482 2025-09-29 00:49:15 2025-09-29 01:15:00 0 days 00:25:45           0.429167
486 2025-09-29 00:50:14 2025-09-29 01:16:00 0 days 00:25:46           0.429444
501 2025-09-29 00:51:50 2025-09-29 01:18:00 0 days 00:26:10           0.436111
527 2025-09-29 00:52:40 2025-09-29 01:23:00 0 days 00:30:20           0.505556
528 2025-09-29 01:00:04 2025-09-29 01:24:00 0 days 00:23:56           0.398889


In [37]:
# Calculate Time betwen Material Requests

# Sort by PLP and datetime_creac
df = df.sort_values(by=['PLP', 'datetime_creac'])

# Calculate time difference in hours between consecutive rows for each PLP
df['time_between_MatReqs'] = df.groupby('PLP')['datetime_creac'].diff().dt.total_seconds() / 3600


# Drop rows where 'time_between_reqs' is NaN
df = df.dropna(subset=['time_between_MatReqs'])

# Verify
print(df.shape)

print(df.head(20))


(56138, 24)
        SFab GLin  UbiLínea      Material  Status  Denominacion Tipo Sum  \
18409    401   20  A07P2_NO  W05FF817157A     1.0  CIMBRA TECHO       NO   
32638    401   20  A07P2_NO  W05FF817157A     1.0  CIMBRA TECHO       NO   
47292    401   20  A07P2_NO  W05FF817157A     1.0  CIMBRA TECHO       NO   
77037    401   20  A07P2_NO  W05FF817157A     1.0  CIMBRA TECHO       NO   
93592    401   20  A07P2_NO  W05FF817157A     1.0  CIMBRA TECHO       NO   
112113   401   20  A07P2_NO  W05FF817157A     1.0  CIMBRA TECHO       NO   
129373   401   20  A07P2_NO  W05FF817157A     1.0  CIMBRA TECHO       NO   
143323   401   20  A07P2_NO  W05FF817157A     1.0  CIMBRA TECHO       NO   
158238   401   20  A07P2_NO  W05FF817157A     1.0  CIMBRA TECHO       NO   
174143   401   20  A07P2_NO  W05FF817157A     1.0  CIMBRA TECHO       NO   
193762   401   20  A07P2_NO  W05FF817157A     1.0  CIMBRA TECHO       NO   
213348   401   20  A07P2_NO  W05FF817157A     1.0  CIMBRA TECHO       NO   


Store Data in a SQL DB

The Data is now prepared for the analysis process done in another Python script.

In this part, this data is stored in a local sql database in one single table.

In [38]:
# Define database name and table name
db_name = "SPA_Data_Analytics.db"  # SQLite databases are usually .db files
table_name = "SPA_Historic_Manual_Requests"


# Create a connection to the SQLite database
conn = sqlite3.connect(db_name)

# Store the DataFrame in the database
df.to_sql(table_name, conn, if_exists='replace', index=False)

# Commit and close the connection
conn.commit()
conn.close()

print(f"Data successfully stored in {db_name}, table: {table_name}")



C:\Users\SPARE06\AppData\Local\Temp\1\ipykernel_3728\3497773135.py:10: UserWarning: the 'timedelta' type is not supported, and will be written as integer values (ns frequency) to the database.
  df.to_sql(table_name, conn, if_exists='replace', index=False)


Data successfully stored in SPA_Data_Analytics.db, table: SPA_Historic_Manual_Requests
